In [6]:
import json
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
tqdm.pandas()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [19]:
file = 'data/result.json'
data = json.load(open(file))
data.keys()

dict_keys(['name', 'type', 'id', 'messages'])

In [20]:
messages = data['messages']

In [21]:
def parse_text(text):
    if isinstance(text, str):
        return text + ' '
    
    all_types = ['bank_card', 'bold', 'code', 'email',
                 'hashtag', 'italic', 'link', 'mention',
                 'phone', 'text_link','underline']
    valid_types = ['bold']
    res = ''
    for t in text:
        if isinstance(t, str):
            res += t + ' '
        elif t['type'] in valid_types:
            res += t['text'] + ' '
    return res

def get_sender(sender):
    return 'soroush' if sender.lower() == 'soroush' else 'erf'

In [22]:
df = pd.DataFrame(data=messages)
df = df[['from', 'text']]

In [23]:
df.drop(df[pd.isna(df['from'])].index, inplace=True)

In [24]:
df['from'] = df.progress_apply(lambda x: get_sender(x['from']), axis=1)

  0%|          | 0/193814 [00:00<?, ?it/s]

In [25]:
df['text'] = df.progress_apply(lambda x: parse_text(x['text']), axis=1)

  0%|          | 0/193814 [00:00<?, ?it/s]

In [26]:
df.drop(df[df['text'] == ''].index, inplace=True)
df.shape

(193425, 2)

In [27]:
history = [{'from':df['from'][0] , 'text': df['text'][0]}]

In [28]:
for f,t in tqdm(zip(df['from'][1:], df['text'][1:])):
    if f == history[-1]['from']:
        history[-1]['text'] += ' ' + t
    else:
        history += [{'from':f, 'text': t}]

0it [00:00, ?it/s]

In [29]:
df2 = pd.DataFrame(data=history)

In [30]:
df2.shape

(103663, 2)

In [31]:
df2.to_csv('processed.csv', index=False)